In [ ]:
import configparser
import cv2  # OpenCV - change image
import math
import pytesseract
import numpy as np

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract"

In [ ]:
config = configparser.ConfigParser()
config.read("config.ini")
BASE_FP = config["FILES"]["default_path"]

In [ ]:
# BASE_FP ..\static\
fp = BASE_FP + "\\changes\\boiling-crab.warped.jpg"

In [ ]:
def resize_with_aspect_ratio(image, width=None, height=None, inter=cv2.INTER_AREA):
    """This is better than our first pass of resize_window"""
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

In [ ]:
imgs = {}

In [ ]:
original = cv2.imread(fp)
imgs["original"] = original

In [ ]:
def noise_removal(img):
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.erode(img, kernel, iterations=1)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    img = cv2.medianBlur(img, 3)
    return img


def remove_noise(image):
    # https://nextgeninvent.com/blogs/7-steps-of-image-pre-processing-to-improve-ocr-using-python-2/
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)

In [ ]:
denoised = noise_removal(original)
imgs["denoised"] = denoised

In [ ]:
def bw(img):
    _, bw_img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    return bw_img

threshold = bw(denoised)
imgs["threshold"] = threshold

In [ ]:
kernel = np.ones((2, 2), np.uint8)
morph = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel)
morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)
imgs["morphology"] = morph

In [ ]:
# Display all imgs into separate windows
for name, img in imgs.items():
    # if "warp" in name:
    #     resize = resize_window(img, 600, 800, "
    #  warp")
    #     cv2.imshow(name, resize)
    # else:
    #     resize = resize_with_aspect_ratio(img, 300, 200)
    #     cv2.imshow(name, resize)
    print(f"{name}: {img.shape}")
    resize = resize_with_aspect_ratio(img, 400, 200)
    cv2.imshow(name, resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
options = "--psm 4"
text = pytesseract.image_to_string(
	# cv2.cvtColor(original, cv2.COLOR_BGR2RGB),
    morph,
	config=options
)

print(text)